In [ ]:
%load_ext autoreload

%autoreload 2
%cd ..

In [2]:
import sys

sys.path.append('.')

In [3]:
import torch
import pandas as pd
import numpy as np

In [8]:
from src.data import BaseData, VectorizedData
from src.vectorize import get_vectorized_data
from src.cluster import KMeansClusterizator
from src.summary import get_cluster_summaries

## EDA

In [ ]:
DATA_PTH = '<DATASET>'
data = pd.read_csv(DATA_PTH, sep=';')
data.head()

In [14]:
class Dataset(BaseData):
    def __init__(self, source=None) -> None:
        super().__init__(source)
        self.data = pd.read_csv(DATA_PTH, sep=';')
        
    def get_chunk(self):
        ...
        
    def get_chunks(self):
        return self.data['comment'].values.tolist()

In [15]:
data = Dataset()

In [16]:
len(data.get_chunks())

300

In [ ]:
data.get_chunks()[-1]

In [19]:
vectorized_data = get_vectorized_data(data)

In [21]:
vectorized_data.save_embeddings()

In [22]:
vectorized_data.load_embeddings()

In [36]:
clusterizator = KMeansClusterizator(vectorized_data)
clusterizator.clusterize()

In [34]:
len(clusterizator.clusters)

11

In [ ]:
clusterizator.get_cluster_summaries()

In [44]:
clusterizator.save()

In [42]:
clusterizator.load()

In [ ]:
cluster = clusterizator.get_cluster_by_id(10)
cluster.chunks

In [47]:
cluster.summary

'Где моя посылка?'

In [ ]:
import umap
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

clusters_dict = {}
for i in range(11):
    embs = clusterizator.get_cluster_by_id(i).embeddings
    clusters_dict[i] = [embs]
    
umap_3d = umap.UMAP(n_neighbors=5, min_dist=0.3, n_components=3)
plt.figure()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for cluster_label, vectors in clusters_dict.items():
    vectors = np.vstack(vectors)
    reduced_data = umap_3d.fit_transform(vectors)
    ax.scatter(reduced_data[:, 0], reduced_data[:, 1], reduced_data[:, 2], label=cluster_label)

plt.title('3D UMAP Visualization')
ax.set_xlabel('Dimension 1')
ax.set_ylabel('Dimension 2')
ax.set_zlabel('Dimension 3')
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
plt.tight_layout()
plt.show()
